In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io 
from sklearn import svm 
import re 
from stemming.porter2 import stem
import nltk, nltk.stem.porter

In [5]:
cd C:\Users\诚明\Desktop\MachineLearning\machine-learning-ex6\ex6

C:\Users\诚明\Desktop\MachineLearning\machine-learning-ex6\ex6


In [6]:
emailcontent=open('emailSample1.txt','r').read()
emailcontent

"> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n"

In [7]:
def preProcess(email):
    email=email.lower()
    email=re.sub('<[^<>]+>', ' ', email)
    email=re.sub('[0-9]+','number',email)
    email=re.sub('(http|https)://[^\s]*','httpaddr',email)          
    email=re.sub('[^\s]+@[^\s]+','emailaddr',email)
    email=re.sub('[$]+','dollar',email)
    return email

In [39]:
def email2TokenList(raw_email):
    stemmer=nltk.stem.porter.PorterStemmer()
    email=preProcess( raw_email )
    tokens=re.split(r'[@=)%(.",_%-:;$*!?#\'<> ]',email)      # Notice that we need to split " " too.
    tokenlist=[]
    for token in tokens:
        token=re.sub('[\W]','',token)
        stemmed=stemmer.stem(token)
        if not len(token): 
            continue
        tokenlist.append(stemmed)
    return tokenlist

In [9]:
" ".join(pd.Series(email2TokenList(emailcontent)))

'anyon know how much it cost to host a web portal well it depend on how mani visitor you re expect thi can be anywher from less than number buck a month to a coupl of dollarnumb you should checkout httpaddr or perhap amazon ecnumb if your run someth big to unsubscrib yourself from thi mail list send an email to emailaddr'

In [10]:
def getVocablist():
    vocab_list={}
    with open('vocab.txt') as f:
        for line in f:
            (val,key)=line.split()
            vocab_list[key]=int(val)
    return vocab_list

In [11]:
def email2VocabIndices(raw_email,vocab_dict):
    tokenlist=email2TokenList(raw_email)
    indlist=[vocab_dict[token] for token in tokenlist if token in vocab_dict]
    return indlist

In [12]:
def email2featurevec(raw_email,vocab_dict):
    indlist=email2VocabIndices(raw_email,vocab_dict)
    result=np.zeros((len(vocab_dict),1))
    result[indlist]=1
    return result

In [13]:
result=email2featurevec(emailcontent,getVocablist())
np.count_nonzero(result)

45

In [14]:
data=scipy.io.loadmat('spamTrain.mat')
X,y=data['X'],data['y']
mat= scipy.io.loadmat('spamTest.mat')
Xtest, ytest = mat['Xtest'], mat['ytest']

In [15]:
fit=svm.SVC(C=0.1,kernel='linear').fit(X,y.flatten())
predtrain=fit.predict(X)
accuracy_train=float(sum(predtrain.reshape((y.size,1))==y)/y.size)
predtest=svm.SVC(C=0.1,kernel='linear').fit(X,y.flatten()).predict(Xtest)
accuracy_test=float(sum(predtest.reshape((ytest.size,1))==ytest)/ytest.size)
print('Accuracy of train samples: %.3f \nAccuracy of test samples: %.3f' % (accuracy_train,accuracy_test))

Accuracy of train samples: 0.998 
Accuracy of test samples: 0.989


In [16]:
coef=pd.DataFrame(fit.coef_.T)
vocab_dict=getVocablist()
vocab_series=pd.Series(vocab_dict)
most_common=vocab_series[coef.nlargest(15,0).index]
least_common=vocab_series[coef.nsmallest(15,0).index]

In [17]:
spam_classifier=pd.DataFrame([most_common.index,least_common.index]).T
spam_classifier.columns=['Most Common','Least Common']
spam_classifier

,Most Common,Least Common
0,our,spamassassin
1,click,the
2,remov,url
3,guarante,wrote
4,visit,date
5,basenumb,list
6,dollar,rpm
7,will,numbertnumb
8,price,user
9,pleas,until


In [43]:
def email2TokenList2(raw_email):
    stemmer=nltk.stem.porter.PorterStemmer()
    email=preProcess( raw_email )
    tokens=re.split('\W',email)      # Notice that we need to split " " too.
    tokenlist=[]
    for token in tokens:
        stemmed=stemmer.stem(token)
        if not len(token): 
            continue
        tokenlist.append(stemmed)
    return tokenlist